In [2]:
import tensorflow as tf
import numpy as np

# Define the XOR input data and labels
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2, input_dim=2, activation='relu'),  # Hidden layer with 2 units and ReLU activation
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with 1 unit and sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_data, y_data, epochs=1000, verbose=0)  # You can adjust the number of epochs as needed

# Evaluate the model
loss, accuracy = model.evaluate(x_data, y_data)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Make predictions
predictions = model.predict(x_data)
print("Predictions:")
for i, x in enumerate(x_data):
    print(f"Input: {x}, Predicted: {predictions[i][0]:.2f}")


1/1 [==============================] - 0s 185ms/step - loss: 0.5523 - accuracy: 0.7500
Loss: 0.5522702932357788, Accuracy: 0.75
1/1 [==============================] - 0s 89ms/step
Predictions:
Input: [0. 0.], Predicted: 0.58
Input: [0. 1.], Predicted: 0.58
Input: [1. 0.], Predicted: 0.58
Input: [1. 1.], Predicted: 0.22


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset (replace with your character dataset)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values

# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # Change 10 to the number of character classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Reshape the data (assuming grayscale images)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=10)  # Change 10 to the number of character classes
y_test = to_categorical(y_test, num_classes=10)

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64)  # Adjust the number of epochs and batch size

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# Make predictions
predictions = model.predict(x_test)


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
938/938 [==============================] - 45s 47ms/step - loss: 0.1832 - accuracy: 0.9429
Epoch 2/5
938/938 [==============================] - 45s 48ms/step - loss: 0.0521 - accuracy: 0.9837
Epoch 3/5
938/938 [==============================] - 44s 47ms/step - loss: 0.0367 - accuracy: 0.9884
Epoch 4/5
938/938 [==============================] - 45s 48ms/step - loss: 0.0287 - accuracy: 0.9911
Epoch 5/5
313/313 [==============================] - 3s 8ms/step - loss: 0.0544 - accuracy: 0.9838
Test accuracy: 0.9837999939918518
313/313 [==============================] - 2s 7ms/step


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import Adam

# Define constants
input_shape = (224, 224, 3)  # Adjust the input size according to your dataset
num_classes = 3  # Replace with the number of individuals/classes to recognize
batch_size = 32
epochs = 10

# Load a pre-trained VGGFace model (VGG16-based)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Create a custom top classifier for face recognition
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Prepare the data (replace with your dataset)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Load your training data and labels
train_generator = train_datagen.flow_from_directory(
    '/content/pig',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

# Train the model
model.fit(train_generator, epochs=epochs)

# Save the trained model
model.save('face_recognition_model.h5')


Found 8 images belonging to 3 classes.
Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 9.0861 - accuracy: 0.3750
Epoch 2/10
1/1 [==============================] - 6s 6s/step - loss: 115.6077 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 6s 6s/step - loss: 109.9589 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 7s 7s/step - loss: 33.7173 - accuracy: 0.7500
Epoch 5/10
1/1 [==============================] - 6s 6s/step - loss: 5.3722e-05 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 6s 6s/step - loss: 7.6635 - accuracy: 0.8750
Epoch 7/10
1/1 [==============================] - 6s 6s/step - loss: 14.9485 - accuracy: 0.8750
Epoch 8/10
1/1 [==============================] - 5s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 7s 7s/step - loss: 1.7128 - accuracy: 0.8750
Epoch 10/10
1/1 [==============================] - 6s 6s/step - loss: 18.1320 - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample text data (you should replace this with your dataset)
text_data = [
    "This is an example of a language model.",
    "Language modeling is important for NLP tasks.",
    "RNNs are commonly used for language modeling.",
    "You can create powerful models using TensorFlow."
]

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences to have the same length
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Split sequences into inputs and labels
input_sequences = tf.constant(input_sequences)
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Build the RNN-based language model
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Train the language model
model.fit(X, y, epochs=100, verbose=1)

# Generate text using the trained model
seed_text = "Language modeling is"
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = model.predict(token_list)
    predicted_word_index = tf.argmax(predicted, axis=-1).numpy()[0]
    predicted_word = tokenizer.index_word[predicted_word_index]
    seed_text += " " + predicted_word

print(seed_text)


Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 3.2206
Epoch 2/100
1/1 [==============================] - 0s 20ms/step - loss: 3.2149
Epoch 3/100
1/1 [==============================] - 0s 15ms/step - loss: 3.2092
Epoch 4/100
1/1 [==============================] - 0s 14ms/step - loss: 3.2033
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 3.1973
Epoch 6/100
1/1 [==============================] - 0s 15ms/step - loss: 3.1910
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 3.1843
Epoch 8/100
1/1 [==============================] - 0s 13ms/step - loss: 3.1771
Epoch 9/100
1/1 [==============================] - 0s 13ms/step - loss: 3.1693
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 3.1608
Epoch 11/100
1/1 [==============================] - 0s 17ms/step - loss: 3.1514
Epoch 12/100
1/1 [==============================] - 0s 14ms/step - loss: 3.1411
Epoch 13/100
1/1 [==============================] -

In [13]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.5 MB/s eta 0:00:00


In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np

# Sample text data and corrected corresponding labels (0 for negative, 1 for positive)
text_data = [
    "I love this product!",
    "This is great. Highly recommended.",
    "Terrible experience. Never buying again.",
    "The service was average.",
]

# Corrected labels: 0 for negative, 1 for positive
labels = [1, 1, 0, 0]

# Tokenize the text data
max_words = 1000  # Maximum number of words to keep in the vocabulary
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(text_data)

# Convert text to sequences and pad them
sequences = tokenizer.texts_to_sequences(text_data)
max_sequence_length = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Define the LSTM-based sentiment analysis model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Convert labels to numpy array
labels = np.array(labels)

# Train the model
model.fit(sequences, labels, epochs=10, verbose=1)

# Evaluate the model on new text data
test_data = [
    "This is a good movie.",
    "I'm not happy with this product.",
    "I can't believe how bad this service is.",
]

test_sequences = tokenizer.texts_to_sequences(test_data)
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

predictions = model.predict(test_sequences)
for i, text in enumerate(test_data):
    sentiment = "positive" if predictions[i] > 0.5 else "negative"
    print(f"'{text}' has a {sentiment} sentiment.")


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.6924 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6850 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6775 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6695 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 27ms/step - loss: 0.6605 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6503 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6385 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6247 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6086 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 18ms/step - loss: 0.5897 - accuracy: 1.0000


1/1 [==============================] - 0s 398ms/step
'This is a good movie.' has a positive sentiment.
'I'm not happy with this product.' has a positive sentiment.
'I can't believe how bad this service is.' has a positive sentiment.


In [26]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

# Sample data for parts of speech tagging
sentences = ["I love natural language processing", "Parts of speech tagging is important"]
tags = ["PRON VERB ADJ NOUN NOUN", "NOUN ADP NOUN NOUN ADJ"]

# Create vocabulary for words and tags
word_vocab = set(" ".join(sentences).split())
tag_vocab = set(" ".join(tags).split())

word_vocab_size = len(word_vocab)
tag_vocab_size = len(tag_vocab)

# Create word and tag dictionaries for mapping between words/tags and their indices
word2idx = {word: idx for idx, word in enumerate(word_vocab)}
idx2word = {idx: word for word, idx in enumerate(word_vocab)}
tag2idx = {tag: idx for idx, tag in enumerate(tag_vocab)}
idx2tag = {idx: tag for tag, idx in enumerate(tag_vocab)}

# Convert text data to numerical sequences
input_sequences = [[word2idx[word] for word in sentence.split()] for sentence in sentences]
target_sequences = [[tag2idx[tag] for tag in tag_sequence.split()] for tag_sequence in tags]

# Find the maximum sequence length
max_sequence_length = max(len(seq) for seq in input_sequences + target_sequences)

# Pad sequences to have the same length
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, padding='post', maxlen=max_sequence_length)
target_sequences = tf.keras.preprocessing.sequence.pad_sequences(target_sequences, padding='post', maxlen=max_sequence_length)

# Define the Seq2Seq model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=word_vocab_size, output_dim=128)(input_layer)
encoder = LSTM(128, return_sequences=True)(embedding_layer)
decoder = Dense(tag_vocab_size, activation='softmax')(encoder)

model = Model(input_layer, decoder)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(input_sequences, target_sequences, epochs=10)

# Perform parts of speech tagging on new text
new_text = "Seq2Seq models are versatile tools for NLP tasks"
new_text_sequence = [word2idx.get(word, 0) for word in new_text.split()]
new_text_sequence = tf.keras.preprocessing.sequence.pad_sequences([new_text_sequence], padding='post', maxlen=max_sequence_length)

predicted_tags = model.predict(new_text_sequence)
predicted_tags = [idx2tag.get(idx, 'UNKNOWN') for idx in np.argmax(predicted_tags, axis=-1)[0]]

print(f"Text: {new_text}")
print(f"Predicted Tags: {' '.join(predicted_tags)}")


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 1.6100 - accuracy: 0.0833
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 1.6008 - accuracy: 0.3333
Epoch 3/10
1/1 [==============================] - 0s 21ms/step - loss: 1.5916 - accuracy: 0.5833
Epoch 4/10
1/1 [==============================] - 0s 24ms/step - loss: 1.5823 - accuracy: 0.5833
Epoch 5/10
1/1 [==============================] - 0s 19ms/step - loss: 1.5726 - accuracy: 0.5833
Epoch 6/10
1/1 [==============================] - 0s 21ms/step - loss: 1.5624 - accuracy: 0.5833
Epoch 7/10
1/1 [==============================] - 0s 22ms/step - loss: 1.5516 - accuracy: 0.5833
Epoch 8/10
1/1 [==============================] - 0s 19ms/step - loss: 1.5399 - accuracy: 0.5833
Epoch 9/10
1/1 [==============================] - 0s 23ms/step - loss: 1.5273 - accuracy: 0.5833
Epoch 10/10
1/1 [==============================] - 1s 662ms/step
Text: Seq2Seq models are versatile tools for NLP tasks
Predicted

In [29]:
import cv2
import numpy as np

# Load a GAN-generated image (replace with your own GAN model)
gan_generated_image = cv2.imread('gan_generated_image.png')

# Define a list of augmentation functions
augmentation_functions = [
    cv2.flip,  # Horizontal flip
    cv2.rotate,  # Rotation
    cv2.GaussianBlur,  # Gaussian blur
    cv2.cvtColor  # Change color space (e.g., from BGR to grayscale)
]

# Apply random augmentations to the GAN-generated image
for augmentation_function in augmentation_functions:
    if augmentation_function == cv2.flip:
        augmented_image = augmentation_function(gan_generated_image, 1)  # Horizontal flip
    elif augmentation_function == cv2.rotate:
        augmented_image = augmentation_function(gan_generated_image, cv2.ROTATE_90_CLOCKWISE)  # Rotation
    elif augmentation_function == cv2.GaussianBlur:
        augmented_image = augmentation_function(gan_generated_image, (5, 5), 0)  # Gaussian blur
    elif augmentation_function == cv2.cvtColor:
        augmented_image = augmentation_function(gan_generated_image, cv2.COLOR_BGR2GRAY)  # Change color space

    # Save the augmented image with a descriptive filename
    filename = f'augmented_{augmentation_function.__name__}.png'
    cv2.imwrite(filename, augmented_image)

# You can further customize the augmentations by adjusting parameters or adding more functions.
